In [55]:
import copy
import random
import math
import time

In [8]:
class AirPlaneData:
    appearance_time: int
    earliest_landing_time: int
    target_landing_time: int
    latest_landing_time: int
    penalty_cost_per_minute_early: int
    penalty_cost_per_minute_late: int

    def __init__(self,
                 appearance_time: int,
                 earliest_landing_time: int,
                 target_landing_time: int,
                 latest_landing_time: int,
                 penalty_cost_per_minute_early: int,
                 penalty_cost_per_minute_late: int):
        self.appearance_time = appearance_time
        self.earliest_landing_time = earliest_landing_time
        self.target_landing_time = target_landing_time
        self.latest_landing_time = latest_landing_time
        self.penalty_cost_per_minute_early = penalty_cost_per_minute_early
        self.penalty_cost_per_minute_late = penalty_cost_per_minute_late


class LandingData:
    num_planes: int
    freeze_time: int
    planes: list[AirPlaneData]
    separation_times: list[list[int]]

    def __init__(self, num_planes: int, freeze_time: int, planes: list[AirPlaneData], separation_times: list[list[int]]):
        self.num_planes = num_planes
        self.freeze_time = freeze_time
        self.planes = planes
        self.separation_times = separation_times


class Solution:
    solution: list[int]
    cost: float

    def __init__(self, current_solution: list[int], cost: float):
        self.solution = current_solution
        self.cost = cost

In [10]:
def load_data(file_path: str) -> LandingData:
    with open(file_path, 'r') as f:
        lines = f.readlines()

    line_idx: int = 0
    len_lines: int = len(lines)

    num_planes, freeze_time = map(int, lines[line_idx].strip().split())
    planes: list[AirPlaneData] = []
    separation_times: list[list[int]] = []


    while line_idx < (len_lines - 1):
        line_idx += 1
        line = list(map(float, lines[line_idx].strip().split()))

        airplane = AirPlaneData(line[0], line[1], line[2], line[3], line[4], line[5])

        separation_time: list[int] = []
        while len(separation_time) < num_planes:
            line_idx += 1
            line = list(map(int, lines[line_idx].strip().split()))
            for x in line:
                separation_time.append(x)

        planes.append(airplane)
        separation_times.append(separation_time)

    return LandingData(num_planes, freeze_time, planes, separation_times)

In [11]:
def objective(current_solution: list[int], landing_data: LandingData) -> int:
    total_penalty: int = 0
    num_planes: int = len(current_solution)
    landing_times: list[int] = [0] * num_planes

    planes: list[AirPlaneData] = landing_data.planes
    separation_times: list[list[int]] = landing_data.separation_times

    for i in range(num_planes):
        plane_idx: int = current_solution[i]
        plane: AirPlaneData = planes[plane_idx]

        landing_time: int = max(plane.earliest_landing_time, plane.appearance_time)

        if i > 0:
            prev_plane_idx: int = current_solution[i - 1]
            landing_time: int = max(landing_time, landing_times[i - 1] + separation_times[prev_plane_idx][plane_idx])

        landing_times[i] = landing_time

        if landing_time < plane.target_landing_time:
            total_penalty += (plane.target_landing_time - landing_time) * plane.penalty_cost_per_minute_early
        elif landing_time > plane.target_landing_time:
            total_penalty += (landing_time - plane.target_landing_time) * plane.penalty_cost_per_minute_late

    return total_penalty

In [14]:
def simulated_annealing(landing_data: LandingData, initial_temp: int, cooling_rate: float, max_iter: int) -> Solution:
    num_planes: int = landing_data.num_planes

    current_solution: list[int] = list(range(num_planes))

    random.shuffle(current_solution)

    current_cost: int = objective(current_solution, landing_data)

    best_solution: list[int] = copy.copy(current_solution)
    best_cost: int = current_cost

    temperature: int = initial_temp

    for _ in range(max_iter):
        new_solution: list[int] = copy.copy(current_solution)

        i, j = random.sample(range(num_planes), 2)
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]

        new_cost: int = objective(new_solution, landing_data)

        if new_cost < current_cost or random.random() < math.exp((current_cost - new_cost) / temperature):
            current_solution = new_solution
            current_cost = new_cost

            if current_cost < best_cost:
                best_solution = copy.copy(current_solution)
                best_cost = current_cost

        temperature *= cooling_rate

    return Solution(best_solution, best_cost)

In [106]:
file_data = "data/airland10.txt"
data = load_data(file_data)

start_time = time.time()

solution = simulated_annealing(data, 1000, 0.95, 50000)

selected_solution = solution
selected_cost = solution.cost

sum_cost: float = selected_cost

for _ in range(9):
    solution = simulated_annealing(data, 1000, 0.95, 50000)
    sum_cost += solution.cost

    if solution.cost < selected_cost:
        selected_solution = copy.copy(solution)
        selected_cost = solution.cost

end_time = time.time()

print("Melhor solução encontrada:", selected_solution.solution)
print("Custo da melhor solução:", '{:>12,.2f}'.format(selected_cost))
print("Custo médio da solução:", '{:>12,.2f}'.format(sum_cost / 10))
print("Tempo médio de execução:", '{:>12,.2f}'.format((end_time - start_time) / 10))

Melhor solução encontrada: [2, 0, 4, 11, 12, 14, 26, 19, 9, 27, 30, 32, 22, 16, 23, 61, 53, 38, 55, 56, 39, 15, 28, 29, 37, 58, 75, 33, 54, 17, 79, 31, 74, 70, 78, 88, 52, 101, 40, 18, 71, 90, 118, 117, 110, 50, 115, 100, 114, 106, 102, 125, 108, 129, 124, 73, 112, 143, 131, 127, 43, 87, 130, 84, 136, 148, 91, 34, 133, 62, 160, 123, 137, 120, 153, 155, 163, 167, 150, 157, 64, 156, 57, 95, 121, 151, 166, 60, 168, 149, 165, 161, 146, 176, 13, 159, 6, 170, 175, 173, 183, 104, 141, 142, 85, 169, 180, 126, 119, 196, 72, 184, 190, 172, 202, 69, 158, 35, 204, 187, 174, 214, 138, 177, 217, 218, 221, 211, 227, 92, 215, 240, 116, 220, 225, 238, 49, 250, 181, 41, 252, 257, 271, 235, 269, 272, 245, 236, 267, 259, 278, 268, 266, 256, 228, 216, 200, 212, 275, 135, 253, 243, 285, 292, 263, 254, 287, 298, 261, 286, 280, 290, 282, 265, 5, 276, 185, 302, 83, 284, 305, 309, 237, 251, 20, 197, 295, 289, 288, 311, 258, 283, 199, 264, 171, 317, 321, 313, 304, 205, 193, 139, 270, 310, 332, 335, 206, 44, 334,

In [38]:
def genetic_objective(current_solution: list[int], planes: list[AirPlaneData]) -> int:
    total_cost: int = 0

    for i, landing_time in enumerate(current_solution):
        plane = planes[i]
        if landing_time < plane.target_landing_time:
            total_cost += (plane.target_landing_time - landing_time) * plane.penalty_cost_per_minute_early
        elif landing_time > plane.target_landing_time:
            total_cost += (landing_time - plane.target_landing_time) * plane.penalty_cost_per_minute_late

    return total_cost

In [108]:
def genetic_algorithm(landing_data: LandingData, population_size: int, generations: int, mutation_rate: float) -> Solution:
    planes: list[AirPlaneData] = landing_data.planes
    num_planes: int = len(planes)

    def generate_initial_population() -> list[list[int]]:
        current_population: list[list[int]] = []

        while len(current_population) < population_size:
            current_solution: list[int] = list(range(num_planes))
            random.shuffle(current_solution)
            current_population.append(current_solution)

        return current_population

    def fitness(current_solution: list[int]) -> int:
        penalty: int = genetic_objective(current_solution, planes)
        for i in range(len(current_solution)):
            for j in range(i + 1, len(current_solution)):
                if abs(current_solution[j] - current_solution[i]) > landing_data.separation_times[i][j]:
                    penalty += 0.01
        return penalty

    def select_parents(current_population: list[list[int]]) -> tuple[list[int], list[int]]:
        current_fitness = [fitness(current_solution) for current_solution in current_population]
        current_probabilities = [f / sum(current_fitness) for f in current_fitness]
        return random.choices(current_population, current_probabilities, k=2)

    def crossover(father: list[int], mother: list[int]) -> tuple[list[int], list[int]]:
        crossover_point: int = random.randint(1, len(father) - 1)
        first_child: list[int] = father[:crossover_point] + mother[crossover_point:]
        second_child: list[int] = mother[:crossover_point] + father[crossover_point:]

        first_child = remove_duplicates(first_child)
        second_child = remove_duplicates(second_child)

        return first_child, second_child

    def remove_duplicates(child: list[int]) -> list[int]:
        all_values = set(range(num_planes))
        present_values = set(child)
        missing_values = list(all_values - present_values)

        seen = set()
        for i in range(len(child)):
            if child[i] in seen:
                child[i] = missing_values.pop()
            else:
                seen.add(child[i])

        return child

    def mutate(current_solution: list[int]) -> list[int]:
        if random.random() < mutation_rate:
            i, j = random.sample(range(num_planes), 2)
            current_solution[i], current_solution[j] = current_solution[j], current_solution[i]

        return current_solution

    population: list[list[int]] = generate_initial_population()
    for generation in range(generations):
        new_population: list[list[int]] = []
        while len(new_population) < population_size:
            parent1, parent2 = select_parents(population)
            child1, child2 = crossover(parent1, parent2)
            mutate(child1)
            mutate(child2)
            new_population.extend([child1, child2])
        population = sorted(new_population, key=fitness, reverse=True)[:population_size]

    best_solution: list[int] = max(population, key=fitness)
    return Solution(best_solution, fitness(best_solution))

In [128]:
file_data = "data/airland10.txt"
data = load_data(file_data)

start_time = time.time()

solution = genetic_algorithm(data, 10, 500, 0.3)

selected_solution = solution
selected_cost = solution.cost

sum_cost: float = selected_cost

for _ in range(9):
    solution = genetic_algorithm(data, 10, 500, 0.3)
    sum_cost += solution.cost

    if solution.cost < selected_cost:
        selected_solution = copy.copy(solution)
        selected_cost = solution.cost

end_time = time.time()

print("Melhor solução encontrada:", selected_solution.solution)
print("Custo da melhor solução:", '{:>12,.2f}'.format(selected_cost))
print("Custo médio da solução:", '{:>12,.2f}'.format(sum_cost / 10))
print("Tempo médio de execução:", '{:>12,.2f}'.format((end_time - start_time) / 10))

Melhor solução encontrada: [60, 148, 21, 125, 47, 72, 97, 58, 92, 141, 10, 118, 113, 109, 76, 56, 64, 15, 11, 12, 95, 106, 104, 129, 30, 143, 130, 123, 131, 101, 99, 98, 14, 37, 149, 17, 147, 71, 112, 22, 44, 90, 3, 117, 54, 13, 68, 94, 107, 18, 126, 121, 144, 120, 40, 45, 65, 62, 69, 88, 23, 140, 33, 51, 19, 66, 115, 132, 43, 2, 55, 59, 119, 145, 102, 53, 146, 38, 46, 139, 87, 84, 142, 135, 82, 127, 25, 116, 31, 29, 5, 61, 103, 35, 79, 122, 7, 85, 83, 114, 28, 133, 110, 57, 36, 24, 52, 0, 89, 70, 111, 9, 91, 93, 63, 108, 86, 20, 34, 49, 78, 27, 77, 75, 6, 41, 73, 80, 105, 138, 100, 124, 32, 39, 50, 137, 4, 1, 134, 26, 42, 67, 136, 74, 96, 81, 48, 8, 16, 128]
Custo da melhor solução: 2,105,014.45
Custo médio da solução: 2,105,327.55
Tempo médio de execução:        22.50


In [122]:
def ant_colony_optimization(landing_data: LandingData,
                            num_ants: int,
                            num_iterations: int,
                            alpha: float,
                            beta: float,
                            evaporation_rate: float,
                            pheromone_init: float) -> Solution:

    num_planes: int = landing_data.num_planes

    pheromone: list[list[float]] = [[pheromone_init for _ in range(num_planes)] for _ in range(num_planes)]

    def calculate_probabilities(current_plane: int, visited: set[int]) -> list[float]:
        probabilities: list[float] = []
        for next_plane in range(num_planes):
            if next_plane not in visited:
                pheromone_value = pheromone[current_plane][next_plane]
                heuristic_value = 1 / (landing_data.planes[next_plane].target_landing_time + 1e-6)
                probabilities.append((pheromone_value ** alpha) * (heuristic_value ** beta))
            else:
                probabilities.append(0)
        total = sum(probabilities)
        return [p / total if total > 0 else 0 for p in probabilities]

    def construct_solution() -> list[int]:
        visited: set[int] = set()
        current_solution: list[int] = []
        current_plane = random.randint(0, num_planes - 1)
        current_solution.append(current_plane)
        visited.add(current_plane)

        while len(current_solution) < num_planes:
            probabilities = calculate_probabilities(current_plane, visited)
            next_plane = random.choices(range(num_planes), weights=probabilities, k=1)[0]
            current_solution.append(next_plane)
            visited.add(next_plane)
            current_plane = next_plane

        return current_solution

    def update_pheromone(current_ants: list[Solution]):
        for i in range(num_planes):
            for j in range(num_planes):
                pheromone[i][j] *= (1 - evaporation_rate)

        for ant in current_ants:
            for idx in range(len(ant.solution) - 1):
                i, j = ant.solution[idx], ant.solution[idx + 1]
                pheromone[i][j] += 1 / ant.cost if ant.cost > 0 else 1

    best_solution: Solution = Solution([], float('inf'))

    for _ in range(num_iterations):
        ants: list[Solution] = []

        for _ in range(num_ants):
            actual_solution = construct_solution()
            cost = objective(actual_solution, landing_data)
            ants.append(Solution(actual_solution, cost))

            if cost < best_solution.cost:
                best_solution = Solution(actual_solution, cost)

        update_pheromone(ants)

    return best_solution

In [127]:
file_data = "data/airland10.txt"
data = load_data(file_data)

start_time = time.time()

solution = ant_colony_optimization(
    landing_data=data,
    num_ants=10,
    num_iterations=100,
    alpha=1.0,
    beta=2.0,
    evaporation_rate=0.1,
    pheromone_init=1.0
)

selected_solution = solution
selected_cost = solution.cost

sum_cost: float = selected_cost

for _ in range(9):
    solution = ant_colony_optimization(
        landing_data=data,
        num_ants=10,
        num_iterations=100,
        alpha=1.0,
        beta=2.0,
        evaporation_rate=0.1,
        pheromone_init=1.0
    )
    sum_cost += solution.cost

    if solution.cost < selected_cost:
        selected_solution = copy.copy(solution)
        selected_cost = solution.cost

end_time = time.time()

print("Melhor solução encontrada:", selected_solution.solution)
print("Custo da melhor solução:", '{:>12,.2f}'.format(selected_cost))
print("Custo médio da solução:", '{:>12,.2f}'.format(sum_cost / 10))
print("Tempo médio de execução:", '{:>12,.2f}'.format((end_time - start_time) / 10))

Melhor solução encontrada: [37, 29, 34, 8, 20, 1, 46, 2, 27, 0, 31, 9, 7, 35, 14, 17, 50, 3, 5, 51, 41, 28, 6, 11, 4, 68, 26, 54, 24, 77, 19, 23, 13, 10, 15, 55, 12, 63, 48, 40, 44, 73, 67, 33, 21, 38, 52, 58, 18, 49, 62, 71, 16, 113, 104, 79, 81, 122, 95, 70, 64, 72, 25, 116, 123, 112, 39, 88, 36, 22, 32, 69, 110, 30, 80, 42, 105, 115, 61, 86, 118, 47, 84, 65, 43, 119, 87, 89, 133, 76, 45, 66, 91, 140, 124, 53, 138, 117, 97, 82, 75, 125, 149, 128, 93, 107, 111, 56, 74, 57, 96, 59, 90, 137, 136, 114, 139, 83, 142, 92, 78, 60, 141, 126, 103, 132, 102, 129, 85, 108, 131, 145, 101, 99, 100, 106, 94, 130, 98, 121, 147, 148, 120, 127, 144, 109, 146, 134, 143, 135]
Custo da melhor solução: 1,191,828.32
Custo médio da solução: 1,367,509.02
Tempo médio de execução:         5.23
